For this exercise we are going to implement an HTTPRouter like you would find in a typical web server using the Trie data structure we learned previously.

There are many different implementations of HTTP Routers such as regular expressions or simple string matching, but the Trie is an excellent and very efficient data structure for this purpose.

The purpose of an HTTP Router is to take a URL path like "/", "/about", or "/blog/2019-01-15/my-awesome-blog-post" and figure out what content to return. In a dynamic web server, the content will often come from a block of code called a handler.

First we need to implement a slightly different Trie than the one we used for autocomplete. Instead of simple words the Trie will contain a part of the http path at each node, building from the root node /

In addition to a path though, we need to know which function will handle the http request. In a real router we would probably pass an instance of a class like Python's SimpleHTTPRequestHandler which would be responsible for handling requests to that path. For the sake of simplicity we will just use a string that we can print out to ensure we got the right handler

We could split the path into letters similar to how we did the autocomplete Trie, but this would result in a Trie with a very large number of nodes and lengthy traversals if we have a lot of pages on our site. A more sensible way to split things would be on the parts of the path that are separated by slashes ("/"). A Trie with a single path entry of: "/about/me" would look like:

(root, None) -> ("about", None) -> ("me", "About Me handler")

We can also simplify our RouteTrie a bit by excluding the suffixes method and the endOfWord property on RouteTrieNodes. We really just need to insert and find nodes, and if a RouteTrieNode is not a leaf node, it won't have a handler which is fine.

Next we need to implement the actual Router. The router will initialize itself with a RouteTrie for holding routes and associated handlers. It should also support adding a handler by path and looking up a handler by path. All of these operations will be delegated to the RouteTrie.

Hint: the RouteTrie stores handlers under path parts, so remember to split your path around the '/' character

Bonus Points: Add a not found handler to your Router which is returned whenever a path is not found in the Trie.

More Bonus Points: Handle trailing slashes! A request for '/about' or '/about/' are probably looking for the same page. Requests for '' or '/' are probably looking for the root handler. Handle these edge cases in your Router.

In [435]:
class RouteTrieNode:
    def __init__(self):
        self.children = {}
        self.handler = "not found handler"

    def insert(self, chars):
        self.children[chars] = RouteTrieNode()

    def is_leaf(self):
        return self.children == {}

class RouteTrie:
    def __init__(self, root_handler):
        self.root = RouteTrieNode()
        self.root.handler = root_handler

    def insert_recursively(self, arr, node, handler):
        """Inserts an array of elements into the trie"""
        # Do nothing if the length of the array is 0
        if len(arr) == 0:
            return

        # If the element is not yet in the Trie, create a new child node for the element
        if arr[0] not in node.children:
            node.children[arr[0]] = RouteTrieNode()

        # If only single element in the array, set handler
        if len(arr) == 1:
            node.children[arr[0]].handler = handler

        # Otherwise, recurse by taking a slice of the original array and setting node as the relevant child
        else:
            self.insert_recursively(arr[1:], node.children[arr[0]], handler)

    def insert(self, arr, handler):
        """Inserts an array of elements into the trie"""
        self.insert_recursively(arr, self.root, handler)

    def find_recursively(self, arr, node):
        """Navigates the trie to find a match for a path given by an array of elements"""
        # If the entire path has been traversed
        if len(arr) == 0:
            return node
        
        # If the path is not found, return not found handler
        if arr[0] not in node.children:
            return None

        # Else, recurse through the tree until all elements in arr have been traversed
        return self.find_recursively(arr[1:], node.children[arr[0]])

    def find(self, arr):
        """Navigates the trie to find a match for a path given by an array of elements"""
        return self.find_recursively(arr, self.root)

class Router:
    def __init__(self, root_handler, not_found_handler):
        """Create a new RouteTrie for holding routes"""
        self.rt = RouteTrie(root_handler)
        self.not_found_handler = not_found_handler

    def add_handler(self, path, handler):
        """Adds handler for a given path into the RouteTrie"""
        self.rt.insert(self.split_path(path), handler)

    def lookup(self, path):
        """Looks up a path in the RouteTrie and returns handler"""
        out_node = self.rt.find(self.split_path(path))
        if out_node:
            return out_node.handler
        return self.not_found_handler
    
    def split_path(self, path):
        """Outputs an array where each element represent a part of the path"""
        path_parts[:] = [part for part in path.split('/') if part != '']
        return path_parts

if __name__ == '__main__':

    # TEST CASE 1: Creating the router and adding /home/about/me handler (creating a fresh new trie)
    router = Router("root handler", "not found handler")
    router.add_handler("/home/about/me", "me handler")
    print('\nTEST CASE 1')
    print(router.lookup("")) # should print 'root handler'
    print(router.lookup("/")) # should print 'root handler'
    print(router.lookup("/home")) # should print 'not found handler'
    print(router.lookup("/home/about")) # should print 'not found handler'
    print(router.lookup("/home/about/")) # should print 'not found handler'
    print(router.lookup("/home/about/me")) # should print 'me handler'

    # TEST CASE 2: The same as above but adding /home/about/me/ instead of /home/about/me
    router = Router("root handler", "not found handler")
    router.add_handler("/home/about/me/", "me handler")
    print('\nTEST CASE 2')
    print(router.lookup("")) # should print 'root handler'
    print(router.lookup("/")) # should print 'root handler'
    print(router.lookup("/home")) # should print 'not found handler'
    print(router.lookup("/home/about")) # should print 'not found handler'
    print(router.lookup("/home/about/")) # should print 'not found handler'
    print(router.lookup("/home/about/me")) # should print 'me handler'

    # TEST CASE 3: Adding /home/about handler (adding handler to node that is not a leaf)
    router.add_handler("/home/about", "about handler")
    print('\nTEST CASE 3')
    print(router.lookup("")) # should print 'root handler'
    print(router.lookup("/")) # should print 'root handler'
    print(router.lookup("/home")) # should print 'not found handler'
    print(router.lookup("/home/about/")) # should print 'about handler' or None if you did not handle trailing slashes
    print(router.lookup("/home/about/me")) # should print 'me handler'

    # TEST CASE 4: Adding /home/about/me/you handler (adding new node and handler which is a leaf)
    router.add_handler("/home/about/me/you", "you handler")
    print('\nTEST CASE 4')
    print(router.lookup("")) # should print 'root handler'
    print(router.lookup("/")) # should print 'root handler'
    print(router.lookup("/home")) # should print 'not found handler'
    print(router.lookup("/home/about")) # should print 'about handler'
    print(router.lookup("/home/about/")) # should print 'about handler'
    print(router.lookup("/home/about/me")) # should print 'me handler'
    print(router.lookup("/home/about/me/you")) # should print 'you handler'

    # TEST CASE 5: Adding empty path and handler
    router.add_handler("", "")
    print('\nTEST CASE 4')
    print(router.lookup("")) # should print 'root handler'
    print(router.lookup("/")) # should print 'root handler'

    # TEST CASE 5: Adding empty path and handler
    router.add_handler("///////////////////////", "Strange handler")
    print('\nTEST CASE 5')
    print(router.lookup("")) # should print 'root handler' - root handler cannot be modified once the router has been made
    print(router.lookup("/")) # should print 'root handler' - root handler cannot be modified once the router has been made


TEST CASE 1
root handler
root handler
not found handler
not found handler
not found handler
me handler

TEST CASE 2
root handler
root handler
not found handler
not found handler
not found handler
me handler

TEST CASE 3
root handler
root handler
not found handler
about handler
me handler

TEST CASE 4
root handler
root handler
not found handler
about handler
about handler
me handler
you handler

TEST CASE 4
root handler
root handler

TEST CASE 5
root handler
root handler
